# Reconstruction of H from a Berger POPC trajectory and order parameter calculation

In this notebook, we'll show you how to do the complete pipeline from downloading a united-atom trajectory file (Berger POPC) from Zenodo, reconstructing hydrogens and calculating the order parameter using buildH. In addition, we show how to plot the calculated order parameters using matplotlib.

Because buildH is intended to be used in the Unix command line in a terminal, many cells in this notebook will launch Unix commands instead of using standard Python cells. We recall, any cell starting with the symbol `!` will launch a Unix command in the parent shell. For example:

In [4]:
!ls

Berger_POPC_25ns_buildH_calcOP.ipynb
endCONF.gro
order_parameter_definitions_MODEL_Berger_POPC.def


Although buildH is usable in principle as a Python module, we do not encourage to do so since it has not been developed to be used that way.

## Checking buildH installation and downloading the trajectory

### buildH installation

First, you have to install buildH. For that you can follow the instructions on the README of the buildH repository.

Before launching this notebook, you should have activated the conda environment from buildH (the `$` below corresponds to you prompt) :

```
$ conda activate buildh

```

If this command worked, you should have got the message `(buildh)` 
at the beginning of your prompt stating that the buildh environment has been properly activated. 

Once done, you have launched this Jupyter notebook :

```
$ jupyter lab
```

At this point, we should be able to launch buildH within this notebook:

In [5]:
!buildH

usage: buildH [-h] [-x XTC] -l LIPID -d DEFOP [-opx OPDBXTC] [-o OUT]
              [-b BEGIN] [-e END] [-pi PICKLE]
              topfile
buildH: error: the following arguments are required: topfile, -l/--lipid, -d/--defop


If you do not get this usage message and have some error, you probably did not activate correctly the buildH conda environment before launching this notebook. One other possibility is that the installation of the buildH environment dit not succeed. In such a case, please refer to buildH installation.

### Trajectory download

Now, let's download a [trajectory](https://doi.org/10.5281/zenodo.13279) of a Berger POPC membrane from Zenodo (taken from NMRlipidsI project). This should take a while for the trr file, so be patient!

In [6]:
!wget https://zenodo.org/record/13279/files/popc0-25ns.trr
!wget https://zenodo.org/record/13279/files/endCONF.gro
!ls

--2021-01-08 19:07:35--  https://zenodo.org/record/13279/files/popc0-25ns.trr
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1712544744 (1.6G) [application/octet-stream]
Saving to: ‘popc0-25ns.trr’

popc0-25ns.trr      100%[===================>]   1.59G  12.3MB/s    in 2m 15s  

2021-01-08 19:09:51 (12.1 MB/s) - ‘popc0-25ns.trr’ saved [1712544744/1712544744]

--2021-01-08 19:09:51--  https://zenodo.org/record/13279/files/endCONF.gro
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1968406 (1.9M) [application/octet-stream]
Saving to: ‘endCONF.gro.1’

endCONF.gro.1       100%[===================>]   1.88M  --.-KB/s    in 0.1s    

2021-01-08 19:09:52 (13.5 MB/s) - ‘endCONF.gro.1’ saved [1968406/1968406]

Berger_POPC_25ns_buildH_calcOP

Now we need one last file. It's a definition file relating generic names for each C-H bond to the atom names in the PDB file. For now we get it from the NMRlipids repo : (**Note: we should update this sentence once the def_files dir is created**)

In [11]:
!wget https://raw.githubusercontent.com/NMRLipids/buildH/master/Berger_POPC_test_case/order_parameter_definitions_MODEL_Berger_POPC.def
!ls

--2021-01-08 19:19:31--  https://raw.githubusercontent.com/NMRLipids/buildH/master/Berger_POPC_test_case/order_parameter_definitions_MODEL_Berger_POPC.def
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2135 (2.1K) [text/plain]
Saving to: ‘order_parameter_definitions_MODEL_Berger_POPC.def’

order_parameter_def 100%[===================>]   2.08K  --.-KB/s    in 0s      

2021-01-08 19:19:31 (6.29 MB/s) - ‘order_parameter_definitions_MODEL_Berger_POPC.def’ saved [2135/2135]

Berger_POPC_25ns_buildH_calcOP.ipynb
endCONF.gro
endCONF.gro.1
order_parameter_definitions_MODEL_Berger_POPC.def
popc0-25ns.trr


Let's have a look to this def file:

In [12]:
!head order_parameter_definitions_MODEL_Berger_POPC.def

gamma1_1 POPC C1  H11
gamma1_2 POPC C1  H12
gamma1_3 POPC C1  H13
gamma2_1 POPC C2  H21
gamma2_2 POPC C2  H22
gamma2_3 POPC C2  H23
gamma3_1 POPC C3  H31
gamma3_2 POPC C3  H32
gamma3_3 POPC C3  H33
beta1 POPC C5  H51


The first column is the generic C-H name, second is the residue (lipid) name, third column the carbon PDB name, last column the hydrogen PDB name.

Great, all files were successfully downloaded, we can now proceed to launching buildH!

## Launching buildH for order parameter calculation

Now, we can launch buildH on this downloaded trajectory. For that we need a few arguments :

- the pdb or gro file corresponding to the system, here `endCONF.gro`;
- the `-l` argument which tells which lipid we want to analyze ( **Note: we need to make a demo of the available lipids that buildH contains (should be in the output of the -h option** ), here `Berger_PLA` ;
- `-d` argument corresponding to the def file we just downloaded, here `order_parameter_definitions_MODEL_Berger_POPC.def` ;
- `-x` argument which is the trajectory file, here `popc0-25ns.trr`.

Please be patient, this will take a while since we have XXX frames (you have time for a coffee ;-) ).

In [14]:
!buildH endCONF.gro -l Berger_PLA -d order_parameter_definitions_MODEL_Berger_POPC.def -x popc0-25ns.trr

Constructing the system...
System has 28526 atoms
Dealing with frame 0 at 0.0 ps.
Dealing with frame 1 at 10.0 ps.
Dealing with frame 2 at 20.0 ps.
Dealing with frame 3 at 30.0 ps.
Dealing with frame 4 at 40.0 ps.
Dealing with frame 5 at 50.0 ps.
Dealing with frame 6 at 60.0 ps.
Dealing with frame 7 at 70.0 ps.
Dealing with frame 8 at 80.0 ps.
Dealing with frame 9 at 90.0 ps.
Dealing with frame 10 at 100.0 ps.
Dealing with frame 11 at 110.0 ps.
^C
Traceback (most recent call last):
  File "/home/fuchs/software/miniconda3/envs/buildh/bin/buildH", line 33, in <module>
    sys.exit(load_entry_point('buildh', 'console_scripts', 'buildH')())
  File "/home/fuchs/papers/buildH_Amelie/buildH/buildh/cli.py", line 232, in main
    core.fast_build_all_Hs_calc_OP(universe_woH, begin, end, dic_OP, dic_lipid, dic_Cname2Hnames)
  File "/home/fuchs/papers/buildH_Amelie/buildH/buildh/core.py", line 528, in fast_build_all_Hs_calc_OP
    Hs_coor = buildHs_on_1C(Cname_position, typeofH2build, helper1_coor

Once completed, we should have a new file called `OPBLABLA` (**Note: to be updated**) containing the order parameter. Note that if you want to call with a different name, please use the option `-o` when launching buildH.

In [15]:
!ls

Berger_POPC_25ns_buildH_calcOP.ipynb
endCONF.gro
endCONF.gro.1
order_parameter_definitions_MODEL_Berger_POPC.def
popc0-25ns.trr


## Analyzing results

First, let's have a look to what this output file `OP_buildH.out` contains.

In [17]:
!head OP_buildH.out

# OP_name            resname atom1 atom2  OP_mean OP_stddev OP_stem
#--------------------------------------------------------------------
gamma1_1             POPC    C1    H11    0.01304  0.12090  0.01069
gamma1_2             POPC    C1    H12    0.00666  0.09279  0.00820
gamma1_3             POPC    C1    H13   -0.01531  0.09141  0.00808
gamma2_1             POPC    C2    H21    0.01511  0.12080  0.01068
gamma2_2             POPC    C2    H22   -0.02050  0.09386  0.00830
gamma2_3             POPC    C2    H23   -0.00660  0.09137  0.00808
gamma3_1             POPC    C3    H31    0.01430  0.11796  0.01043
gamma3_2             POPC    C3    H32   -0.00513  0.08670  0.00766


For each C-H we get the mean order parameter (OP_mean), its standard deviation (OP_stddev), and its standard error of the mean (OP_stem). This OP_stem is calculated by averaging over the trajectory for each C-H, and then we averare over residues (**note: check the formula -> I guess we kind of calculate the stdev over each C-H mean (this latter averaged over frames)**)